In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Embedding,GRU,Dense
from keras.optimizers import Adam
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score

In [3]:
dataset =pd.read_csv("reviews.txt",delimiter='\t',names=['Reviews','Sentiment'])

In [4]:
dataset.head()

,Reviews,Sentiment
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.,1


In [5]:
X=dataset.drop(['Sentiment'],axis=1)
y=dataset.drop(['Reviews'],axis=1)

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state=0)

In [7]:
X.head()

,Reviews
0,So there is no way for me to plug it in here i...
1,"Good case, Excellent value."
2,Great for the jawbone.
3,Tied to charger for conversations lasting more...
4,The mic is great.


In [8]:
X_series = dataset.iloc[:,0]

In [9]:
num_words = 2000

In [10]:
tokenizer = Tokenizer(num_words = num_words)

In [11]:
%%time
tokenizer.fit_on_texts(X_series)

CPU times: total: 0 ns
Wall time: 38.3 ms


In [12]:
if num_words is None:
    num_words = len(tokenizer.word_index)

In [13]:
tokenizer.word_index

{'the': 1,
 'i': 2,
 'and': 3,
 'it': 4,
 'is': 5,
 'a': 6,
 'this': 7,
 'to': 8,
 'phone': 9,
 'my': 10,
 'for': 11,
 'of': 12,
 'not': 13,
 'with': 14,
 'very': 15,
 'great': 16,
 'was': 17,
 'on': 18,
 'in': 19,
 'that': 20,
 'good': 21,
 'have': 22,
 'you': 23,
 'product': 24,
 'quality': 25,
 'had': 26,
 'headset': 27,
 'works': 28,
 'battery': 29,
 'but': 30,
 'as': 31,
 'sound': 32,
 'so': 33,
 'are': 34,
 'well': 35,
 'one': 36,
 'all': 37,
 'use': 38,
 'ear': 39,
 'has': 40,
 'would': 41,
 'work': 42,
 'from': 43,
 'your': 44,
 'like': 45,
 'be': 46,
 'me': 47,
 'case': 48,
 'if': 49,
 'than': 50,
 "i've": 51,
 "don't": 52,
 'no': 53,
 'excellent': 54,
 'up': 55,
 'time': 56,
 "it's": 57,
 'after': 58,
 'price': 59,
 'recommend': 60,
 'does': 61,
 'really': 62,
 '2': 63,
 'at': 64,
 'or': 65,
 'best': 66,
 'out': 67,
 'only': 68,
 'service': 69,
 'get': 70,
 'when': 71,
 'nice': 72,
 "i'm": 73,
 'also': 74,
 'too': 75,
 'just': 76,
 'any': 77,
 'new': 78,
 'love': 79,
 'these'

In [14]:
x_train = X_train['Reviews']
x_test = X_test['Reviews']
Y_test = y_test['Sentiment']
Y_test.head()

993    0
859    0
298    0
553    1
672    0
Name: Sentiment, dtype: int64

In [15]:
x_train_tokens = tokenizer.texts_to_sequences(x_train)

In [16]:
type(x_train) 

pandas.core.series.Series

In [17]:
np.array(x_train_tokens[15])

array([404, 164,  34, 159,  76, 135, 104])

In [18]:
x_test_tokens = tokenizer.texts_to_sequences(x_test)

In [19]:
np.array(x_test_tokens[15])

array([180,  96,   4,   5, 135,   3, 874,  11,   1,  59,   2, 875])

In [20]:
num_tokens = [len(tokens) for tokens in x_train_tokens + x_test_tokens]
num_tokens = np.array(num_tokens)

In [21]:
np.mean(num_tokens)

10.29

In [22]:
np.max(num_tokens)

30

In [23]:
pad = 'pre'
max_tokens = 20

In [24]:
x_train_pad = pad_sequences(x_train_tokens, maxlen = max_tokens, padding = pad, truncating = pad)

In [25]:
x_test_pad = pad_sequences(x_test_tokens, maxlen = max_tokens, padding = pad, truncating = pad)

In [26]:
x_train_pad.shape

(750, 20)

In [27]:
x_test_pad.shape

(250, 20)

In [28]:
np.array(x_train_tokens[1])

array([167,  21,  19,   1, 248,  30,   7,  48,  17,   6, 601, 310])

In [29]:
x_train_pad[1]

array([  0,   0,   0,   0,   0,   0,   0,   0, 167,  21,  19,   1, 248,
        30,   7,  48,  17,   6, 601, 310])

## Building the RNN

In [30]:
model = Sequential()

In [31]:
model.add(Embedding(input_dim = num_words,
                   output_dim = 8,
                   input_length = max_tokens,
                   name = 'layer_embedding'))

In [32]:
model.add(GRU(units = 16, return_sequences = True))

In [33]:
model.add(GRU(units = 8, return_sequences = True))

In [34]:
model.add(GRU(units = 4))

In [35]:
model.add(Dense(1, activation = 'sigmoid'))

In [36]:
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [37]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 layer_embedding (Embedding)  (None, 20, 8)            16000     
                                                                 
 gru (GRU)                   (None, 20, 16)            1248      
                                                                 
 gru_1 (GRU)                 (None, 20, 8)             624       
                                                                 
 gru_2 (GRU)                 (None, 4)                 168       
                                                                 
 dense (Dense)               (None, 1)                 5         
                                                                 
Total params: 18,045
Trainable params: 18,045
Non-trainable params: 0
_________________________________________________________________


In [38]:
model.fit(x_train_pad, y_train, epochs =150, batch_size=32)

Epoch 1/150
24/24 [==============================] - 11s 33ms/step - loss: 0.6931 - accuracy: 0.5093
Epoch 2/150
24/24 [==============================] - 1s 35ms/step - loss: 0.6869 - accuracy: 0.5907
Epoch 3/150
24/24 [==============================] - 1s 35ms/step - loss: 0.6664 - accuracy: 0.7307
Epoch 4/150
24/24 [==============================] - 1s 34ms/step - loss: 0.5834 - accuracy: 0.7560
Epoch 5/150
24/24 [==============================] - 1s 34ms/step - loss: 0.4192 - accuracy: 0.8573
Epoch 6/150
24/24 [==============================] - 1s 35ms/step - loss: 0.2570 - accuracy: 0.9440
Epoch 7/150
24/24 [==============================] - 1s 34ms/step - loss: 0.1814 - accuracy: 0.9613
Epoch 8/150
24/24 [==============================] - 1s 34ms/step - loss: 0.1380 - accuracy: 0.9813
Epoch 9/150
24/24 [==============================] - 1s 34ms/step - loss: 0.1265 - accuracy: 0.9787
Epoch 10/150
24/24 [==============================] - 1s 34ms/step - loss: 0.0950 - accuracy: 0.994

24/24 [==============================] - 1s 34ms/step - loss: 0.0037 - accuracy: 1.0000
Epoch 83/150
24/24 [==============================] - 1s 34ms/step - loss: 0.0036 - accuracy: 1.0000
Epoch 84/150
24/24 [==============================] - 1s 35ms/step - loss: 0.0036 - accuracy: 1.0000
Epoch 85/150
24/24 [==============================] - 1s 36ms/step - loss: 0.0035 - accuracy: 1.0000
Epoch 86/150
24/24 [==============================] - 1s 38ms/step - loss: 0.0034 - accuracy: 1.0000
Epoch 87/150
24/24 [==============================] - 1s 38ms/step - loss: 0.0033 - accuracy: 1.0000
Epoch 88/150
24/24 [==============================] - 1s 37ms/step - loss: 0.0033 - accuracy: 1.0000
Epoch 89/150
24/24 [==============================] - 1s 37ms/step - loss: 0.0032 - accuracy: 1.0000
Epoch 90/150
24/24 [==============================] - 1s 38ms/step - loss: 0.0031 - accuracy: 1.0000
Epoch 91/150
24/24 [==============================] - 1s 37ms/step - loss: 0.0031 - accuracy: 1.0000
Epo

In [39]:
result = model.predict(x_test_pad)

8/8 [==============================] - 2s 8ms/step


In [40]:
result = np.round(result, decimals = 0)

In [41]:
result = pd.DataFrame(result)

In [42]:
result.head()

,0
0,0.0
1,0.0
2,0.0
3,1.0
4,0.0


In [43]:
cm = confusion_matrix(y_test, result)

In [44]:
cm

array([[ 88,  32],
       [ 30, 100]], dtype=int64)

In [45]:
from sklearn.metrics import accuracy_score, classification_report

In [46]:
ac = accuracy_score(y_test, result)
ac

0.752

In [47]:
cf = classification_report(y_test, result)
cf

'              precision    recall  f1-score   support\n\n           0       0.75      0.73      0.74       120\n           1       0.76      0.77      0.76       130\n\n    accuracy                           0.75       250\n   macro avg       0.75      0.75      0.75       250\nweighted avg       0.75      0.75      0.75       250\n'